<div align="center">
 <img src="https://raw.githubusercontent.com/matheusmota/dataviz2018/master/resources/images/logo_facens_pos.png" width="100x">
    <h3>Aprendizado de Máquina, Redes Neurais e Deep Learning</h3>  
    <h4>Classificação de cursos Udemy</h4>
</div>


* **203066**  - Evandro Bertolucci
* **110257**  - João Victor Carvalho
* **203071**  - Louise Constantino
* **203087**  - Luiza Constantino
* **203019**  - Murilo Piva
* **203263**  - Rafael Henrique

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

import warnings
import os

# Performance metric
from sklearn.metrics import f1_score

df = pd.read_csv("udemy_courses.csv", encoding='latin-1')
warnings.filterwarnings(action="ignore")

In [2]:
#Limpar coluna URL
df['clean_url'] = df['url'].str.replace('https://www.udemy.com/','')
df['clean_url'] = df['clean_url'].str.replace('/','')
df['clean_url'] = df['clean_url'].str.replace('-',' ')

In [3]:
#Criar coluna juntando as colunas 'Course Title' e 'Clean Url'
df['course_url'] = df['course_title'] + ' ' + df['clean_url']

In [4]:
#Criar novo DataFrame apenas com as colunas necessárias para analise
dfCursos = df[['course_title', 'url', 'course_url', 'subject']]

In [5]:
#Função para limpar campos
def clean_text(text):
    # remove barra invertida-apóstrofo
    text = re.sub("\'", "", text)
    # remove tudo, exceto alfabetos
    text = re.sub("[^a-zA-Z]"," ",text)
    # remove os espaços em branco
    text = ' '.join(text.split())
    # converter texto em minúsculas
    text = text.lower()
    
    return text

In [6]:
#Aplica a função de limpar o campo na coluna 'Course Url'
dfCursos['clean_course'] = dfCursos['course_url'].apply(lambda x: clean_text(x))

In [7]:
#Download do pacote de palavras irrelevantes da biblioteca NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Luiza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#Função para remover as palavras irrelevantes
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

In [9]:
#Removendo as palavras irrelevantes da coluna 'Clean Course'
dfCursos['clean_course'] = dfCursos['clean_course'].apply(lambda x: remove_stopwords(x))

In [10]:
# criando vocabulário
def palavrasVocabulario(qtdLinhaPorRotulo, dadosUdemy_Tratados):
    vocabulario = []
    colunas_usadasModelo = ['course_title', 'url']

    for subject in dadosUdemy_Tratados['subject'].unique():
        for row in dadosUdemy_Tratados[dadosUdemy_Tratados['subject'] == subject]['clean_course'][:qtdLinhaPorRotulo]:
            for p in row.lower().split():
                p = p.replace(' ','')
                if p not in vocabulario:
                    vocabulario.append(p)
    
    return vocabulario

In [11]:
# agora é necessário representar cada frase usando o vocabulário
def Gerar_Amostras(dfCursos, vocabulario):
    amostras = []
    for t in list(dfCursos['clean_course']):
        amostra = []
        for p in vocabulario:
            if p in t.lower().replace('.','').split():
                amostra.append(1)
            else:
                amostra.append(0)

        amostras.append(amostra)

    return np.array(amostras)

In [12]:
#vou pegar todas as linhas, len(dtCursos)
vocabulario = palavrasVocabulario(len(dfCursos),dfCursos)
amostras = Gerar_Amostras(dfCursos, vocabulario)  

In [13]:
voc = ''
with open("vocabulario.txt", "w") as text_file:
    for x in vocabulario:
        text_file.write(x + ",")

In [14]:
X, y = dfCursos.iloc[:,2:3].values, dfCursos.iloc[:,3:4].values

In [15]:
model = ComplementNB()
model.fit(amostras, y)

ComplementNB()

In [16]:
from joblib import dump

dump(model, 'udemy.joblib') 

['udemy.joblib']

In [17]:
vo = []
with open("vocabulario.txt", encoding='utf8') as txt:
    for x in txt:
        li = x.split(',')  
        for h in li:
            if h not in vo and len(h) > 0:
                vo.append(h)